In [ ]:
import gensim
import numpy as np
import scipy
import scipy.spatial
import MeCab
import nltk
import xlrd
import string
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

mecab = MeCab.Tagger("-Owakati")

In [ ]:
# Select all words in the data file and compute the vocabulary. 
# Write the cross-lingual word embeddings for those words to a separate file.
# This will speed up loading word embeddings and save memory.

data_files = ["../data/olddata.xlsx", "../data/newdata.xlsx"]
vocab = set()
for fname in data_files:
    trans_data = xlrd.open_workbook(fname)
    sheet = trans_data.sheet_by_index(0)  
    for l in range(1, sheet.nrows):
        # tokenise Japanese texts
        rows = sheet.row_values(l, 0, sheet.ncols)
        token_ja = mecab.parse(rows[0].lower())
        vocab = vocab.union(set(token_ja.strip().split()))    
        # tokenise English texts
        vocab = vocab.union(set(nltk.word_tokenize(rows[1].lower())))

stop_words = ['(', ')', '[', ']', '@', '•', '`', '-', '❚❚', '●', '（√',  '×', '。', '＠']
add_words = ['I', 'like', 'hate', 'cat', 'cats', 'dog', 'dogs', 'banana', '好き', '嫌い', '猫', '犬', '私']
vocab = vocab - set(stop_words)
vocab = vocab.union(set(add_words))
print("No of unique words in the vocabulary = %d" % len(vocab))

# write the vocabulary to a file for debugging purposes
with open("../data/vocab.txt", 'w') as vocab_file:
    for word in vocab:
        vocab_file.write("%s\n" % word)

# Lets select the cross-lingual word embeddings for those words in the vocabulary.
cross_in_embeds_fname = "../data/ja2en.txt"
cross_out_embeds_fname = "../data/ja2en.sel"
first_line = True

with open(cross_in_embeds_fname) as cross_in:
    with open(cross_out_embeds_fname, 'w') as cross_out:
        for line in cross_in:
            if first_line:
                dim = int(line.split()[1])
                cross_out.write("%d %d\n" % (len(vocab), dim))
                first_line = False
            elif line.split()[0].lower() in vocab:
                cross_out.write(line)

In [ ]:
# Load the cross-lingual word embeddings.
large_embeddings = gensim.models.KeyedVectors.load_word2vec_format('../data/ja2en.txt')
small_embeddings = gensim.models.KeyedVectors.load_word2vec_format('../data/ja2en.sel')

In [ ]:
embeddings = large_embeddings

In [ ]:
def clean_text(s):
    stop_words = ['(', ')', '[', ']', '@', '•', '`', '-', '❚❚', '●', '（√',  '×', '。', '＠']
    for ch in stop_words:
        s = s.replace(ch, ' ')
    return s

In [ ]:
def mwmd(source, target):
    # remove words that are not in the vocabulary from source and target.
    source = list(filter(lambda x: x in embeddings, source))
    target = list(filter(lambda x: x in embeddings, target))
     
    n = len(source)
    m = len(target)
    
    # compute distances between words
    C = np.zeros((n, m), dtype=float)
    for i in range(n):
        for j in range(m):
            first, second = embeddings[source[i]],  embeddings[target[j]]
            
            first_norm, second_norm = np.linalg.norm(first), np.linalg.norm(second)
            if first_norm > 0:
                first = first / first_norm
            if second_norm > 0:
                second = second / second_norm
            
            C[i,j] = scipy.spatial.distance.euclidean(first, second)
    
    # Initialise variables
    x = np.zeros(n + n*m, dtype=float)
    T = x[n:].reshape(n,m)
    y = x[:n]
    
    c = np.zeros_like(x)
    c[:n] = 1.0
    
    # Inequality constraints
    b_ub = np.zeros(n*m, dtype=float)
    A_ub = np.zeros((n*m, n + n*m), dtype=float)    
    for p in range(n*m):
        for q in range(n + n*m):
            if p % n == q:
                A_ub[p, q % n] = -1.0
            if (p // n) + 2 * (p % n) + n == q:
                A_ub[p,q] = C[p % n, p // n]    
    
    # Equality constraints for Eq. 5 (Columns in T must be stochastic)
    CA_eq = np.zeros((n, n + n*m), dtype=float)
    Cb_eq = np.ones(n, dtype=float)
    for p in range(n):
        for q in range(n + m*p, n + m + m*p):
            CA_eq[p,q] = 1.0
            
    # Equality constraints for Eq. 4 (Rows in T must be stochastic)
    RA_eq = np.zeros((m, n + n*m), dtype=float)
    Rb_eq = np.ones(m, dtype=float)
    for p in range(m):
        for q in range(n, n + n*m):
            if p == (q - n) % m:
                RA_eq[p,q] = 1.0
    
    
    res = scipy.optimize.linprog(c, A_ub, b_ub, CA_eq, Cb_eq, method='interior-point', options={'maxiter':10000})
    #res = scipy.optimize.linprog(c, A_ub, b_ub, method='simplex')
    status = {0 : "Optimization terminated successfully",
              1 : "Iteration limit reached",
              2 : "Problem appears to be infeasible",
              3 : "Problem appears to be unbounded",
              4 : "Serious numerical difficulties encountered"}
    if res.status > 0:
        print("\x1b[31m %s \x1b[0m" % status[res.status])
    
    if res.status == 2:
        # Infeasible problem. Drop equality constrains and try again.
        res = scipy.optimize.linprog(c, A_ub, b_ub, method='interior-point') 
        distance_y = np.sum(res.x[:n])
        distance_TC = C.flatten().dot(res.x[n:])
        return (distance_y, 2)        
    
    if res.status == 0:        
        print("No of iterations to optimisation = %d" % res.nit)
        # objective is the sum of y_i.
        distance_y = np.sum(res.x[:n])
        #print("sum y = ", distance_y)
        distance_TC = C.flatten().dot(res.x[n:])
        #print("sum TC = %f" % distance_TC)
        return (distance_y, res.status)
    else:
        return (0, res.status) 

In [ ]:
# Process a dataset, predict similarities and save to a file.
trans_data = xlrd.open_workbook('../data/newdata.xlsx')  
sheet = trans_data.sheet_by_index(0)  
scores = []
outputs = []
distancesst = []
distancests = []
for l in range(1, sheet.nrows):
    rows = sheet.row_values(l, 0, sheet.ncols)
    source = list(set(mecab.parse(clean_text(rows[0]).lower().strip('\n')).split()))
    target = list(set(nltk.word_tokenize(clean_text(rows[1]).lower().strip())))
    #res1 = sms(source, target)
    #distancesst.append(res1)
    res1 = mwmd(source, target)
    vals_t = -1 if res1[1] > 0 else res1[0]
    distancesst.append(vals_t)
    
    res2 = mwmd(target, source)
    valt_s = -1 if res2[1] > 0 else res2[0]
    distancests.append(valt_s)

distancesst = np.array(distancesst)
distancests = np.array(distancests)
print("stot",distancesst)
print("ttos",distancests)
scores = distancesst + distancests


scores = np.array(scores)
max_val = np.max(scores)
print("max val", max_val)
scores = 1.0 - (scores / max_val)

with open("../data/pred-sims-mwmd.csv", "w") as out_file:
    for val in scores:
        #print(val)
        out_file.write("%f\n" % val)